In [27]:
import numpy as np
import math
import random as rn
import pandas as pd
from library import Map, GradientDescent
import time
import tensorflow as tf
from sklearn.linear_model import SGDClassifier
from random import randrange
from sklearn.linear_model import LinearRegression

In [80]:
'''Downloading dataset'''
dataset = pd.read_csv("/Users/alessandrococcia/Downloads/ObesityDataSet.csv")
dataset = dataset.sample(frac=1) #shuffle sample in the training set

'''mapping delle stringhe'''

#dataset = pd.get_dummies(dataset, drop_first=True).astype(float)

'''normalization'''
dataset = (dataset-dataset.min())/(dataset.max()-dataset.min())

'''Inserimento colonna di bias'''
dataset.insert(0, "Bias", np.ones(len(dataset)), True) #Bias row



TRAIN_TEST_SPLIT_PERCENTAGE = 0.9
dataset_training = dataset[:int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE)]
dataset_test = dataset[int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE):]

In [97]:
class Map:
    def __init__(self):
        self.count = 0
        self.count_dataset = {}
        self.d = {}
        self.dataset_dictionary = {}
        self.matrice = None
        self.dataset = None
        len = None
    
    def mappingElement(self, string):
        if type(string) is str:
            if self.d.get(string) is not None:
                return self.d.get(string)
            else:
                self.count = self.count+1
                self.d[string] = self.count
                return self.count
        return float(string)
    
    def mappingMatrix(self, matrice):
        self.matrice = None
        self.matrice = np.array(matrice)
        len = self.matrice.shape
        for g in range(len[0]):
            for j in range(len[1]):
                self.matrice[g][j] = self.mappingElement(self.matrice[g][j])
        return np.float64(self.matrice.copy())
    
    def mappingElementDataset(self, value, colonna):
        if type(value) is str:
            if self.dataset_dictionary[colonna].get(value) is not None:
                return self.dataset_dictionary[colonna].get(value)
            else:
                if self.count_dataset.get(colonna) is None:
                    self.count_dataset[colonna] = 0
                    self.dataset_dictionary[colonna][value] = self.count_dataset[colonna]
                    return self.count_dataset[colonna]
                self.count_dataset[colonna] += 1
                self.dataset_dictionary[colonna][value] = self.count_dataset[colonna]
                return self.count_dataset[colonna]
        return value
    
    def mappingDataset(self, dataset):
        self.dataset = dataset
        colonne = dataset.columns
        length = len(dataset)
        for column in colonne:
            self.dataset_dictionary[column] = {}
            for index in range(length):
                dataset.at[index, column] = self.mappingElementDataset(dataset.at[index, column], column)
        return self.dataset

    def MSE(self, a, b):
        return np.mean((np.square(a - b)))
    def RMSE(self, a, b):
        return math.sqrt(np.mean(np.square(a - b)))
    def MAE(self, a, b):
        return np.mean(abs(a-b))
    

class DecisionTree:
    def __init__(self, outputstring, positive, negative): 
        self.outputstring = outputstring
        self.positive = positive
        self.negative = negative
        self.nodes = []
        self.label = None
        self.root = False

    def LearnDecisionTree(self, examples, attributes, parent_examples, column_values):
        self.root = True
        self.LearnDecisionTreeAux_(examples, attributes, parent_examples, column_values)

    def LearnDecisionTreeAux_(self, examples, attributes, parent_examples, column_values):
        same_classification = self.SameClassification(examples)    
        
        if len(examples.loc[:, examples.columns != self.outputstring]) == 0:
            return self.PluralityValue(parent_examples)
        elif same_classification is not False:
            return same_classification
        elif len(attributes) == 0: 
            return self.PluralityValue(examples)
        else:
            bestattribute = self.Importance(attributes, examples)
            self.label = bestattribute #Seleziono l'attributo migliore
            
            for value in self.Values1(bestattribute, column_values):
                remainingexamples = self.Examples(bestattribute, examples, value)
                tree = DecisionTree(self.outputstring, self.positive, self.negative)
                attributes_left = self.PopListValue(attributes.copy(), bestattribute)                
                subtree = tree.LearnDecisionTreeAux_(remainingexamples.loc[:, remainingexamples.columns != bestattribute], attributes_left, examples, column_values)
                self.nodes.append((value, subtree))
        return self
        
    def Importance(self, attributes, examples): 
        max = -1
        ret = None
        for a in attributes:
            loc = self.Gain(examples, a)
            if loc > max:
                max = loc
                ret = a
        return ret
        
    def B(self, q):
        if q == 1 or q == 0:
            return 0
        return -(q*math.log2(q)+(1-q)*math.log2(1-q))
    
    def Remainder(self, examples, attribute, p, n):
        sum = 0
        
        for v in self.Values2(attribute, examples):
            if type(v) is not str and math.isnan(float(v)):
                pk = len(examples.loc[(examples[self.outputstring] == self.positive) & (examples[attribute].isnull())])
                nk = len(examples.loc[(examples[self.outputstring] == self.negative) & (examples[attribute].isnull())])
            else:
                pk = len(examples.loc[(examples[self.outputstring] == self.positive) & (examples[attribute] == v)])
                nk = len(examples.loc[(examples[self.outputstring] == self.negative) & ((examples[attribute] == v))])
            
            partial = (pk+nk)*self.B(pk/(pk+nk))
            sum += partial
        return (1/(p+n))*sum
    
    def Gain(self, examples, attribute):
        p = len(examples.loc[examples[self.outputstring] == self.positive])
        n = len(examples.loc[examples[self.outputstring] == self.negative])
        b = self.B(p/(p+n))
        r = self.Remainder(examples, attribute, p, n)
        return (b-r)
        
    def PluralityValue(self, parent_examples): 
        '''Selects the most common ouput value among a set of examples, breaking ties randomly'''
        value, max = [], 0
        d = self.CreateDictionary(parent_examples)
        for key in d.keys():
            if d.get(key) > max:
                max = d.get(key)
                value = []
                value.append(key)
            elif d.get(key) == max:
                '''se ci sono più massimi li metto tutti in una lista dalla quale ne
                    selezionerò uno randomicamente
                '''
                value.append(key)
        return rn.choice(value)
        
    def SameClassification(self, examples): 
        if (len(examples) == 0):
            return False
        d = self.CreateDictionary(examples)
        if (len(d.keys()) == 1):
            return list(d.keys())[0]
        return False
    def CreateDictionary(self, examples):
        examples = examples[self.outputstring].tolist()
        d = {}
        for i in range(len(examples)):
            if d.get(examples[i]) is None:
                d[examples[i]] = 1
            else:
                d[examples[i]] += 1
        return d

    def Values1(self, attribute, dictionary): 
        return list(dictionary.get(attribute))
    
    def Values2(self, attribute, examples):
        return examples[attribute].unique()

    def Examples(self, attribute, examples, value):
        exs = examples.loc[examples[attribute] == value]
        return exs
    def PopListValue(self, lista, value):
        if value not in lista:
            return None
        lista.remove(value)
        return lista 
    def PrintDecisionTree(self, count):
        print(" "*count, self.label)
        for elem in self.nodes:
            if type(elem[1]) is not DecisionTree:
                print("    "*(count+1), elem[0], " --> ", elem[1])
            else:
                elem[1].PrintDecisionTree(count+1)

    def Prediction(self, input):
        for i in range(len(self.nodes)):
            x = input[self.label].values[0]
            if type(self.nodes[i][0]) is not str and math.isnan(float(self.nodes[i][0])):
                if type(x) is not str and math.isnan(float(x)):
                    if type(self.nodes[i][1]) is DecisionTree:
                        return self.nodes[i][1].Prediction(input)
                    return self.nodes[i][1]
            elif self.nodes[i][0] == x:
                if type(self.nodes[i][1]) is DecisionTree:
                    return self.nodes[i][1].Prediction(input)
                return self.nodes[i][1]
        return None

class GradientDescent:
    def __init__(self, learningrate, epochs, parameters_length, output_string, batch_size): 
        self.learningrate = learningrate
        self.epochs = epochs
        self.parameters_length = parameters_length
        self.parameters = None
        self.output_string = output_string
        self.len_totale = None
        self.batch_size = batch_size

    def SGD(self, training_set):
        
        self.parameters = np.ones(self.parameters_length)
        
        input_totale = training_set.loc[:len(training_set), training_set.columns != self.output_string]
        output_totale = training_set.loc[:len(training_set), training_set.columns == self.output_string]
        self.len_totale = len(training_set)

        error = self.MSE(self.prediction(input_totale), output_totale)

        for i in range(self.epochs):
            
            batch = training_set.sample(n=self.batch_size)
            input = batch.loc[:, training_set.columns != self.output_string]
            output = batch.loc[:, training_set.columns == self.output_string]
            
            self.learningrate = 10000/((10000+i))
            gradient = self.gradient(input, output)
            self.SGD_aux(gradient)
            
            if (not (i%10000)):
                print("Epoca: \t", i)
                print("MSE: ", self.MSE(self.prediction(input_totale), output_totale))
                
        return self.parameters

    def SGD_aux(self, gradient):
        for f in range(self.parameters_length):
            self.parameters[f] = self.parameters[f]-(self.learningrate*gradient[f][0])

    def gradient(self, input, output): 
        output = np.array(output)
        prediction = self.prediction(input)
        error = prediction - output
        coefficient = input.shape[0]/self.len_totale
        return np.dot(coefficient*input.T, error)

    def prediction(self, input):
        return np.dot(input, self.parameters)

    def MSE(self, prediction, output):
        output = np.array(output)
        return np.mean((output - prediction)**2)
    def RMSE(self, prediction, output):
        return math.sqrt(np.mean((output - prediction)**2))


In [103]:
gd = GradientDescent(1, 1000000, dataset_training.shape[1]-1, 'Weight', 1)

start = time.time()
pesi = gd.SGD(dataset_training)
end = time.time()
print(end-start, pesi)

#'''sklearn'''
#X = dataset_training.loc[:, dataset_training.columns != 'Weight']
#Y = dataset_training.loc[:, dataset_training.columns == 'Weight']
#
#clf = SGDClassifier(max_iter = 1, tol=1e-3)
#clf.fit(X, Y)



Epoca: 	 0
MSE:  94.24147309169841
Epoca: 	 10000
MSE:  0.22890763703815303
Epoca: 	 20000
MSE:  0.19356253868187628
Epoca: 	 30000
MSE:  0.1827545283193402
Epoca: 	 40000
MSE:  0.17702964846606886
Epoca: 	 50000
MSE:  0.17361941235674144
Epoca: 	 60000
MSE:  0.17074111877952053
Epoca: 	 70000
MSE:  0.16841270253935714
Epoca: 	 80000
MSE:  0.16661877261073582
Epoca: 	 90000
MSE:  0.16500713172085618
Epoca: 	 100000
MSE:  0.1635470102406828
Epoca: 	 110000
MSE:  0.16243225631595523
Epoca: 	 120000
MSE:  0.16138431931451483
Epoca: 	 130000
MSE:  0.16056409287257625
Epoca: 	 140000
MSE:  0.15966304366315728
Epoca: 	 150000
MSE:  0.15860629403363835
Epoca: 	 160000
MSE:  0.1580507687271559
Epoca: 	 170000
MSE:  0.1573484501682887
Epoca: 	 180000
MSE:  0.1564470653662937
Epoca: 	 190000
MSE:  0.15579435039827638
Epoca: 	 200000
MSE:  0.15521490057351608
Epoca: 	 210000
MSE:  0.154598423366599
Epoca: 	 220000
MSE:  0.15432764047659145
Epoca: 	 230000
MSE:  0.15390727520290415
Epoca: 	 240000

In [124]:
X_test = dataset_test.loc[:, dataset_test.columns != 'Weight']
X_test = np.array(X_test)
Y_effettivo = np.array(dataset_test['Weight'])

X = dataset_training.loc[:, dataset_training.columns != 'Weight']
Y = np.array(dataset_training.loc[:, dataset_training.columns == 'Weight'])

Y_predetto = np.array(np.matmul(X_test, pesi))
Y_predetto_training = np.array(np.matmul(X, pesi))

map = Map()

print("MSE: ", map.MSE(Y, Y_predetto_training))
print("RMSE: ", map.RMSE(Y, Y_predetto_training))
print("MAE: ", map.MAE(Y, Y_predetto_training))


#print("MSE: ", map.MSE(Y_effettivo, Y_predetto))
#print("RMSE: ", map.RMSE(Y_effettivo, Y_predetto))
#print("MAE: ", map.MAE(Y_effettivo, Y_predetto))
#
#'''sklearn'''

#
#reg = LinearRegression().fit(X, Y)
#scikit_predict = reg.predict(X_test)
#print("MSE_sklearn: ", map.MSE(Y_effettivo, scikit_predict))
#print("RMSE_sklearn: ", map.RMSE(Y_effettivo, scikit_predict))
#print("MAE_sklearn: ", map.MAE(Y_effettivo, scikit_predict))

reg = LinearRegression().fit(X, Y)
scikit_predict = reg.predict(X)

print("MSE_sklearn: ", map.MSE(Y_effettivo, scikit_predict))
print("RMSE_sklearn: ", map.RMSE(Y_effettivo, scikit_predict))
print("MAE_sklearn: ", map.MAE(Y_effettivo, scikit_predict))

MSE:  0.13194763756535088
RMSE:  0.36324597391485414
MAE:  0.2903156196847688
MSE_sklearn:  0.07227572911243803
RMSE_sklearn:  0.2688414572056141
MAE_sklearn:  0.2184483581987949
